## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from pathlib import Path
START_DIR = Path('/content/drive/MyDrive/Colab_Notebooks/Fine_tuning_HF_SD_model_using_DreamBooth_LoRA')
# START_DIR = '.'
os.chdir(START_DIR)


In [ ]:
%pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
START_DIR

PosixPath('/content/drive/MyDrive/Colab_Notebooks/Fine_tuning_HF_SD_model_using_DreamBooth_LoRA')

## Fine-tuning

In [ ]:
import json
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS
from pathlib import Path
from slugify import slugify
import subprocess
import shutil
from waitress import serve

In [ ]:
parameters_file = START_DIR / "images_parameters.json"

In [ ]:
training_prompt = "jelly in a glass"
sdiff_model_type = "regular"
### For production $USER_ID is generated from Login page on Frontend ###
### Here the *user_id* variable is received from the frontend form for testing purposes only - REPLACE it for production  ###
user_id = "dprhyl"
### For production MODIFY the line above ###############################################################


In [ ]:
# Retrieve available image parameters from JSON file
with open(parameters_file) as json_file:
    sdiff_dict = json.load(json_file)
sdiff_model = sdiff_dict["sdiff_models"][sdiff_model_type]["name"].split("/")[-1]
resolution = sdiff_dict["sdiff_models"][sdiff_model_type]["resolution"]

# convert training prompt to a valid directory name
prompt_as_path = slugify(training_prompt)

# construct file paths
MODEL_PATH = START_DIR / "HF_models" / sdiff_model
USER_DIR = START_DIR / "Users" / user_id
LORA_DIR = USER_DIR / "custom_models"
TRAINING_DATA_PATH = USER_DIR / "training_data" / prompt_as_path
LR_SCHEDULER = "constant"

# print some debug info
print("*** FINETUNING ***")
print("User: ", user_id)
print("Training Prompt: ", training_prompt)
print("Stable Diffusion Model: ", sdiff_model)
print("Loading training images from: ", TRAINING_DATA_PATH)


In [ ]:
 # Launch train_dreambooth_lora.py with command line parameters
print("Training is running, ETA = 4 min. for *512 px*, ETA = 9 min. for *768 px* ")
subprocess.run(
    [
        "accelerate",
        "launch",
        "train_dreambooth_lora.py",
        f"--pretrained_model_name_or_path={MODEL_PATH}",
        f"--instance_data_dir={TRAINING_DATA_PATH}",
        f"--output_dir={LORA_DIR}",
        f"--instance_prompt={training_prompt}",
        f"--resolution={resolution}",
        f"--train_batch_size=1",
        f"--gradient_accumulation_steps=1",
        f"--checkpointing_steps=250",
        f"--learning_rate=1e-4",
        f"--lr_scheduler={LR_SCHEDULER}",
        f"--lr_warmup_steps=0",
        f"--max_train_steps=500",
        f"--validation_prompt={training_prompt}",
        f"--validation_epochs=50",
        f"--seed=0",
        f"--gradient_checkpointing",
        f"--use_8bit_adam",
        f"--enable_xformers_memory_efficient_attention",
    ],
    capture_output=True,
    text=True,
)


In [ ]:
# Rename LoRA model
created_model = LORA_DIR / "pytorch_lora_weights.bin"
LORA_PATH = created_model.rename(LORA_DIR / f"{prompt_as_path}__{sdiff_model}.bin")
print("Model saved as ", LORA_PATH)

# Cleaning up - Remove intermediate checkpoints
shutil.rmtree(LORA_DIR / "checkpoint-250")
shutil.rmtree(LORA_DIR / "checkpoint-500")

print("*** Finetuning finished successfully! ***\n")

In [ ]:
# Run app in debugging (vanilla Flask on default "127.0.0.1:5000")

app.run(debug=True)

In [ ]:
# Run app in production (Waitress)
serve(app, host=localhost, port=8080)